# side model feature selection

In [2]:
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(7996, 3447)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [3]:
import numpy as np

label_long = np.load("data/side_label_long.npy")
label_short = np.load("data/side_label_short.npy")

print(label_long.shape)
print(label_short.shape)

len_gap = len(label_long) - len(df_features)

label_long = label_long[len_gap:]
label_short = label_short[len_gap:]

print(label_long.shape)
print(label_short.shape)


(8060,)
(8060,)
(7996,)
(7996,)


In [4]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, label_long)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: williams_r (最大重要性: 0.0353)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3447个特征中选择了689个，舍弃了2758个
✅ 选择的特征: ['williams_r', 'conv_1', 'hasbrouck_lambda_ddt', 'voss_filt_dt', 'adx_7_dt', 'vwap_dt_lag1', 'ehlers_early_onset_trend_ddt', 'natr_lag19', 'cmma', 'ehlers_early_onset_trend_ddt_lag1', 'roofing_filter_ddt', 'stc_dt', 'roofing_filter_dt', 'vwap_dt', 'trendflex_dt_lag1', 'cmma_lag1', 'adaptive_cci_dt', 'roofing_filter_ddt_lag1', 'stc_dt_lag1', 'evenbetter_sinewave_short_dt', 'pfe_dt_lag19', 'bandpass_dt', 'williams_r_lag1', 'voss', 'adaptive_stochastic_dt', 'price_variance_ratio_ddt', 'stc', 'fisher_dt', 'ehlers_early_onset_trend_dt', 'acc_swing_index_dt_lag1', 'fisher_dt_lag1', 'acc_swing_index_dt', 'evenbetter_sinewave_short_dt_lag1', 'reactivity', 'cmma_dt_lag2', 'evenbetter_sinewave_long_dt', 'natr_lag15', 'norm_on_balance_volume', 'cmma_dt', 'cmma_dt_lag3', 'voss_dt_lag1', 'adx_7_ddt', 'cmma_dt_lag1', 'reactivity_dt', 'trendflex_dt', 'trendflex_dt_lag2', 'bekker_parkinson_vol_dt', 'comb_spectrum_pwr_20', 'fisher_dt_lag2', 'highpass_bp_dt',

In [5]:
side_res_long = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_long > 0).sum())
side_res_long

295


williams_r                 0.035290
roofing_filter_ddt         0.033681
vwap_dt_lag1               0.030814
trendflex_dt_lag1          0.030221
roofing_filter_ddt_lag1    0.025303
                             ...   
comb_spectrum_pwr_27       0.000000
comb_spectrum_pwr_28       0.000000
comb_spectrum_pwr_29       0.000000
comb_spectrum_pwr_30       0.000000
williams_r_ddt_lag19       0.000000
Length: 3447, dtype: float64

In [6]:
selector = RFCQSelector()
selector.fit(df_features, label_short)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: adaptive_cci_dt (最大重要性: 0.0300)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3447个特征中选择了689个，舍弃了2758个
✅ 选择的特征: ['adaptive_cci_dt', 'acc_swing_index_lag14', 'bekker_parkinson_vol_dt_lag1', 'stc', 'ehlers_early_onset_trend_ddt', 'natr_dt', 'natr_dt_lag2', 'forecast_oscillator_lag3', 'cmma_dt_lag1', 'reactivity_dt', 'reactivity_dt_lag1', 'voss_filt_dt', 'natr_dt_lag1', 'cmma_lag2', 'ehlers_early_onset_trend_ddt_lag1', 'evenbetter_sinewave_short_dt_lag1', 'natr', 'roofing_filter_dt', 'adaptive_stochastic_dt', 'norm_on_balance_volume', 'roofing_filter_ddt_lag1', 'acc_swing_index_lag12', 'cmma_dt', 'aroon_diff', 'evenbetter_sinewave_long_dt', 'fisher_dt_lag13', 'vwap_dt_lag1', 'cmma_lag1', 'fisher_dt_lag2', 'natr_lag8', 'natr_dt_lag3', 'cmma_dt_lag2', 'bandpass_dt', 'williams_r', 'roofing_filter_ddt', 'natr_lag1', 'reactivity_dt_lag2', 'cmma_dt_lag3', 'norm_on_balance_volume_dt_lag1', 'aroon_diff_dt_lag1', 'adaptive_bp_lead', 'williams_r_dt_lag3', 'trendflex_dt_lag1', 'acc_swing_index_dt_lag1', 'evenbetter_sinewave_long_lag1', 'stc_dt_lag2', 'vwap_dt', 'sw

In [7]:
side_res_short = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_short > 0).sum())
side_res_short


68


roofing_filter_ddt_lag1              0.03
adaptive_stochastic_dt               0.03
adaptive_cci_dt                      0.03
natr_dt_lag1                         0.03
ehlers_early_onset_trend_ddt         0.03
                                     ... 
comb_spectrum_dom_cycle_ddt_lag13    0.00
comb_spectrum_dom_cycle_ddt_lag14    0.00
comb_spectrum_dom_cycle_ddt_lag15    0.00
comb_spectrum_dom_cycle_ddt_lag16    0.00
williams_r_ddt_lag19                 0.00
Length: 3447, dtype: float64

In [8]:
import json

feature_info = {
    "side": {
        "long": [],
        "short": [],
    },
    "meta": {
        "meta": [],
        "model_long": [],
        "model_short": [],
    },
}

feature_long = side_res_long[side_res_long > 0].index.tolist()
feature_short = side_res_short[side_res_short > 0].index.tolist()

feature_info["side"]["long"] = feature_long
feature_info["side"]["short"] = feature_short


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(df_features.shape)
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(7996, 3447)
(7996,)


adx_7                               0
mod_rsi_lag17                       0
mod_rsi_lag19                       0
mod_rsi_dt_lag1                     0
mod_rsi_dt_lag2                     0
                                   ..
comb_spectrum_dom_cycle_ddt_lag5    0
comb_spectrum_dom_cycle_ddt_lag6    0
comb_spectrum_dom_cycle_ddt_lag7    0
comb_spectrum_dom_cycle_ddt_lag8    0
williams_r_ddt_lag19                0
Length: 3447, dtype: int64

In [2]:
from strategies.BinanceBtcEntropyBarV1.config import (
    SIDE_LONG,
    SIDE_SHORT,
    get_side_model,
)

model_long = get_side_model(False, "long")
model_short = get_side_model(False, "short")

res_long = model_long.predict(df_features[SIDE_LONG])
res_short = model_short.predict(df_features[SIDE_SHORT])

assert df_features.shape[0] == len(res_long) == len(res_short)

df_features["model_long"] = res_long
df_features["model_short"] = res_short

In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: adx_7_dt (最大重要性: 0.0400)
➤ 计算特征冗余度...
➤ 总计选择689个特征 (已选择1个，还需选择688个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/688 [00:00<?, ?特征/s]


✅ 特征选择完成：从3449个特征中选择了689个，舍弃了2760个
✅ 选择的特征: ['adx_7_dt', 'acc_swing_index_lag17', 'forecast_oscillator_lag3', 'price_change_oscillator', 'hasbrouck_lambda_ddt', 'fti_ddt', 'reactivity', 'price_variance_ratio_dt', 'bekker_parkinson_vol_ddt_lag2', 'natr_lag11', 'voss', 'corwin_schultz_estimator_lag1', 'amihud_lambda_dt_lag2', 'natr_lag2', 'adx_14_dt', 'natr_ddt_lag1', 'hasbrouck_lambda_lag7', 'kyle_lambda_lag10', 'price_variance_ratio_dt_lag2', 'bekker_parkinson_vol_ddt', 'vwap_dt', 'acc_swing_index_lag18', 'price_change_oscillator_dt', 'evenbetter_sinewave_long', 'corwin_schultz_estimator', 'model_short', 'iqr_ratio_dt', 'bekker_parkinson_vol_dt_lag2', 'natr_lag16', 'trendflex_dt_lag1', 'hasbrouck_lambda_lag2', 'price_variance_ratio_dt_lag1', 'adx_14_dt_lag1', 'natr_lag14', 'cmma_lag1', 'bekker_parkinson_vol', 'vwap_lag1', 'natr_lag7', 'cmma', 'adx_7_dt_lag1', 'natr_lag1', 'natr_lag6', 'natr_dt_lag1', 'model_long', 'bekker_parkinson_vol_lag1', 'roll_impact_lag16', 'acc_swing_index_lag1

In [4]:
(meta_res > 0).sum()

76

In [5]:
import json
from pathlib import Path

feature_info = Path("strategies/BinanceBtcEntropyBarV1/feature_info.json")
if feature_info.exists():
    with open(feature_info, "r") as f:
        feature_info = json.load(f)

feature_info["meta"]["meta"] = []
feature_info["meta"]["model_long"] = []
feature_info["meta"]["model_short"] = []

meta_features = meta_res[meta_res > 0].index.tolist()
for k in meta_features:
    if k == "model_long":
        feature_info["meta"]["model_long"].append(k)
    elif k == "model_short":
        feature_info["meta"]["model_short"].append(k)
    else:
        feature_info["meta"]["meta"].append(k)


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)